In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
H1 = 200
H2 = 800
g1 = 10
g2 = 0.05


A = np.array([
    [1/H1/g1 + 1/H1/g2, -1/H1/g2],
    [-1/H2/g2, 1/H2/g2]
])

In [ ]:
1/(H1*H2/(H1+H2)*g2)

In [ ]:
1/((H1+H2)*g1)

In [ ]:
eigvals, eigvect = np.linalg.eig(A)

In [ ]:
A @ eigvect[:,0] / eigvect[:,0]

In [ ]:
D = np.diag(eigvals)

In [ ]:
eigvect[:,0]

In [ ]:
eigvect[:,1]

In [ ]:
eigvect

In [ ]:
np.linalg.inv(eigvect) @ A @ eigvect

In [ ]:
alpha = 1
psi = np.array([1,alpha])

In [ ]:
np.linalg.inv(eigvect) @ psi

In [ ]:
A @ psi

In [ ]:
1 / H1 / g1